In [1]:
# import packages
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import time

from contextlib import redirect_stdout
from tensorflow.keras import datasets, layers, models

2024-04-09 10:37:53.091967: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Datasets
train_path = '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS3/train'
test_path =  '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS3/test'
csv_names = [
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/base_model.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/oneofeach_model.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/twoofeach_model.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/threeofeach_model.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/fourofeach_model.csv'
]
summary_names = [
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/base_model_summary.txt',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/oneofeach_model_summary.txt',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/twoofeach_model_summary.txt',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/threeofeach_model_summary.txt',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/model_depth/fourofeach_model_summary.txt'
]

def preprocess(example):
    image = example['image']
    image.set_shape([16, 16, 1])
    label = example['label']
    label = tf.one_hot(label, depth=100)  # One-hot encode the labels
    return image, label

def format_set(train_set, test_set):
    # format and cache
    train_set_formatted = train_set.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train_set_formatted = train_set_formatted.cache()
    train_set_formatted = train_set_formatted.batch(512)
    train_set_formatted = train_set_formatted.prefetch(tf.data.AUTOTUNE)
    test_set_formatted = test_set.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    test_set_formatted = test_set_formatted.batch(512)
    test_set_formatted = test_set_formatted.cache()
    test_set_formatted = test_set_formatted.prefetch(tf.data.AUTOTUNE)
    return train_set_formatted, test_set_formatted

In [3]:
# Keeping time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

In [4]:
# Model taken from example (https://www.tensorflow.org/tutorials/images/cnn)
base_model = models.Sequential()
base_model.add(layers.Conv2D(32, (2, 2), activation='relu')) # input_shape removed, (3, 3) -> (2, 2)
base_model.add(layers.MaxPooling2D((2, 2)))
base_model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
base_model.add(layers.MaxPooling2D((2, 2)))
base_model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
base_model.add(layers.Flatten())
base_model.add(layers.Dense(64, activation='relu'))
base_model.add(layers.Dense(100))  # changed from 10 to 100, due to amount of classes

# 1 of each
oneofeach_model = models.Sequential()
oneofeach_model.add(layers.Conv2D(32, (2, 2), activation='relu')) # input_shape removed, (3, 3) -> (2, 2)
oneofeach_model.add(layers.MaxPooling2D((2, 2)))
oneofeach_model.add(layers.Flatten())
oneofeach_model.add(layers.Dense(100))  # changed from 10 to 100, due to amount of classes

# 2 of each
twoofeach_model = models.Sequential()
twoofeach_model.add(layers.Conv2D(32, (2, 2), activation='relu')) # input_shape removed, (3, 3) -> (2, 2)
twoofeach_model.add(layers.MaxPooling2D((2, 2)))
twoofeach_model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
twoofeach_model.add(layers.MaxPooling2D((2, 2)))
twoofeach_model.add(layers.Flatten())
twoofeach_model.add(layers.Dense(64, activation='relu'))
twoofeach_model.add(layers.Dense(100))  # changed from 10 to 100, due to amount of classes

# 3 of each
threeofeach_model = models.Sequential()
threeofeach_model.add(layers.Conv2D(32, (2, 2), activation='relu')) # input_shape removed, (3, 3) -> (2, 2)
threeofeach_model.add(layers.MaxPooling2D((2, 2)))
threeofeach_model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
threeofeach_model.add(layers.MaxPooling2D((2, 2)))
threeofeach_model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
threeofeach_model.add(layers.MaxPooling2D((2, 2)))
threeofeach_model.add(layers.Flatten())
threeofeach_model.add(layers.Dense(64, activation='relu'))
threeofeach_model.add(layers.Dense(64, activation='relu'))
threeofeach_model.add(layers.Dense(100))  # changed from 10 to 100, due to amount of classes

# 4 of each
fourofeach_model = models.Sequential()
fourofeach_model.add(layers.Conv2D(32, (2, 2), activation='relu')) # input_shape removed, (3, 3) -> (2, 2)
fourofeach_model.add(layers.MaxPooling2D((2, 2)))
fourofeach_model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
fourofeach_model.add(layers.MaxPooling2D((2, 2)))
fourofeach_model.add(layers.Conv2D(64, (2, 2), activation='relu')) # (3, 3) -> (2, 2)
fourofeach_model.add(layers.MaxPooling2D((2, 2)))
fourofeach_model.add(layers.Conv2D(64, (1, 1), activation='relu')) # (3, 3) -> (2, 2)
fourofeach_model.add(layers.MaxPooling2D((1, 1)))
fourofeach_model.add(layers.Flatten())
fourofeach_model.add(layers.Dense(64, activation='relu'))
fourofeach_model.add(layers.Dense(64, activation='relu'))
fourofeach_model.add(layers.Dense(64, activation='relu'))
fourofeach_model.add(layers.Dense(100))  # changed from 10 to 100, due to amount of classes


dnn_models = [
    base_model,
    oneofeach_model,
    twoofeach_model,
    threeofeach_model,
    fourofeach_model
]
    

In [5]:
def compile_model(model):
    model.compile(
        optimizer='adamax',
        loss=tf.keras.losses.BinaryFocalCrossentropy(),
        metrics=['accuracy']
    )
    return model

def myprint(s):
    with open('modelsummary.txt','a') as f:
        print(s, file=f)

In [6]:
# List of optimizers to iterate over
for i in range(len(csv_names)):    
    # fetch datasets
    train = tf.data.Dataset.load(train_path)
    test  = tf.data.Dataset.load(test_path)
    train, test = format_set(train, test)

    # create model
    model = dnn_models[i]
    model = compile_model(model)
    model.build((None, 16, 16, 1))
    model.summary()

    # Train
    history = model.fit(train, epochs=250, validation_data=test, callbacks=[time_callback])

    # Convert the history.history dict to a pandas DataFrame
    hist_df = pd.DataFrame(history.history)

    # Add epoch times
    hist_df['time'] = time_callback.times

    # Save model summary
    with open(summary_names[i], 'w') as f:
        for layer in model.layers:
            print(type(layer).__name__, file=f)

        # print params
        total_params         = model.count_params()
        trainable_params     = sum([tf.size(w_matrix).numpy() for w_matrix in model.trainable_weights])
        non_trainable_params = sum([tf.size(w_matrix).numpy() for w_matrix in model.non_trainable_weights])
        #optimizer_params     = sum([tf.size(w_matrix).numpy() for w_matrix in model.optimizer.weights])
        print(f'Total params: {total_params}', file=f)
        print(f'Trainable params: {trainable_params}', file=f)
        print(f'Non-trainable params: {non_trainable_params}', file=f)
        # print(f'Optimizer params: {optimizer_params} \n', file=f)
    
    # Save to csv
    hist_df.to_csv(csv_names[i])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 15, 15, 32)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 6, 6, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 2, 2, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │         6,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,812 (186.77 KB)

 Trainable params: 47,812 (186.77 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 15, 15, 32)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 6, 6, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 2, 2, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 1, 1, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 100)            │         6,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,004 (187.52 KB)

 Trainable params: 48,004 (187.52 KB)

 Non-trainable params: 0 (0.00 B)

98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.0105 - loss: 0.0665 - val_accuracy: 0.0116 - val_loss: 0.0302
